In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By  # HTML요소 찾기 위한 모듈
from selenium.webdriver.common.keys import Keys  # 키보드 입력 제어
from selenium.webdriver.chrome.service import Service  # Chrome드라이버 실행을 위한 서비스 모듈
# from webdriver_manager.chrome import CharomeDriverManager  # Chrome 드라이버 자동 설치

import time
import tqdm
import pandas as pd

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 브라우저 창 없이 실행 (백그라운드)
options.add_argument("--no-sandbox")  # 리눅스 환경에서 필요
options.add_argument("--disable-dev-shm-usage")  # 메모리 사용 최소화

In [ ]:
# 크롬 드라이버 실행
# service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=options)

# 요소탐색 기다림 시간 설정
driver.implicitly_wait(5)  # 5초 설정

# 카카오맵 검색 url 설정
url = 'https://map.kakao.com'
driver.get(url)
print(driver.title)  # 현재 페이지 출력

# 브라우저에서 검색창 찾기 및 검색어 입력
search_box = driver.find_element(By.ID, 'search.keyword.query')  # 검색창 요소 찾기
search_box.send_keys("카페")  # 검색어 입력
search_box.send_keys(Keys.RETURN)  # 엔터키 입력
print(driver.title, "카페 검색")  # 현재 페이지 출력

# 팝업 재거
time.sleep(1)
driver.execute_script('document.elementFromPoint(10, 10).click();')

# 장소 카테고리 버튼 클릭
place_btn = driver.find_element(By.CSS_SELECTOR, ".mainHeader > ul.sub > li.option1 > a") 
place_btn.click()  # 장소 버튼 클릭
print(driver.title, "장소 버튼 클릭")  # 현재 페이지 출력

time.sleep(3)

In [ ]:
# 데이터 저장
cafe_kakaoMaplist = []

set_page = 0
for page in range(50):

    print("페이지:", page)
    click_page = page - set_page

    # 검색 결과 리스트 가져오기
    cafes = driver.find_elements(By.CSS_SELECTOR, '.placelist > .PlaceItem')  # 각 카페 정보가 담긴 요소 찾기

    for cafe in cafes:

        try:
            name = cafe.find_element(By.CSS_SELECTOR, 'strong.tit_name > .link_name').text  # 카페 이름
            addr_load = cafe.find_element(By.CSS_SELECTOR, '[data-id = "address"]').text  # 카페 도로명 주소
            addr_lot = cafe.find_element(By.CSS_SELECTOR, '[data-id = "otherAddr"]').text  # 카페 지번 주소

            try:
                rating_score = cafe.find_element(By.CSS_SELECTOR, 'em[data-id = "scoreNum"]').text  # 카페 평점
                rating_num = cafe.find_element(By.CSS_SELECTOR, 'a[data-id = "numberofscore"]').text  # 카페 평가수
            except:
                rating_score = "No Rating"
                rating_num = 0
            
            cafe_kakaoMaplist.append([name, addr_load, addr_lot, rating_score, rating_num])

        except Exception as e:
            print("Error:", e)  # 오류 발생시 Error메세지 출력

    print(f"{page+1} 페이지 저장 완료")

    # 다음 페이지 버튼 클릭
    try:
        page_btn = driver.find_element(By.XPATH, f'//*[@id="info.search.page.no{click_page+2}"]')
        page_btn.click()  # 다음 페이지 버튼 클릭
        print(f"{page+2} 페이지 이동")
        print(f"클릭한 xpaht 숫자: {click_page+2}")

        time.sleep(1)
        
    except:
        try:
            print("> 버튼클릭 시도")
            set_page += 5
            next_btn = driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]')
            next_btn.click()  # > 버튼 클릭
            print("> 버튼 클릭")

            time.sleep(1)

        except:    
            print("마지막 페이지")
            break

driver.quit()  # 크롬 드라이버 종료

In [ ]:
# 데이터프레임 생성 및 
df_cafe = pd.DataFrame(cafe_kakaoMaplist, columns=['name', 'addr_load', 'addr_lot', 'rating_score', 'rating_num'])

# csv파일 저장
df_cafe.to_csv("cafe_kakaoMaplist.csv", index=False, encoding='utf-8-sig')

In [ ]:
# 카카오맵 내저장 장소
# 데이터 저장
cafe_kakaoMap_mylist = []

set_page = 0
for page in range(1):

    print("페이지:", page)
    click_page = page - set_page

    # 검색 결과 리스트 가져오기
    cafes = driver.find_elements(By.CSS_SELECTOR, '.list_detail > .FavoriteDetailItem')  # 각 카페 정보가 담긴 요소 찾기

    for cafe in cafes:

        try:
            name = cafe.find_element(By.CSS_SELECTOR, '.info_directory > .directory_info > .tit_directory > .txt_directory > .link_txt').text  # 카페 이름
            addr_load = cafe.find_element(By.CSS_SELECTOR, '.info_directory > .directory_info > .desc_directory > .desc_region').text  # 카페 도로명 주소
            # addr_lot = cafe.find_element(By.CSS_SELECTOR, '[data-id = "otherAddr"]').text  # 카페 지번 주소

            try:
                rating_score = cafe.find_element(By.CSS_SELECTOR, 'em[data-id = "scoreNum"]').text  # 카페 평점
                rating_num = cafe.find_element(By.CSS_SELECTOR, 'a[data-id = "numberofscore"]').text  # 카페 평가수
            except:
                rating_score = "No Rating"
                rating_num = 0
            
            cafe_kakaoMap_mylist.append([name, addr_load, rating_score, rating_num])

        except Exception as e:
            print("Error:", e)  # 오류 발생시 Error메세지 출력

    print(f"{page+1} 페이지 저장 완료")

    # 다음 페이지 버튼 클릭
    try:
        page_btn = driver.find_element(By.XPATH, f'//*[@id="info.my.favorite.page.no{click_page+2}"]')
        page_btn.click()  # 다음 페이지 버튼 클릭
        print(f"{page+2} 페이지 이동")
        print(f"클릭한 xpaht 숫자: {click_page+2}")

        time.sleep(1)
        
    except:
        try:
            print("> 버튼클릭 시도")
            set_page += 5
            next_btn = driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]')
            next_btn.click()  # > 버튼 클릭
            print("> 버튼 클릭")

            time.sleep(1)

        except:    
            print("마지막 페이지")
            break

# driver.quit()  # 크롬 드라이버 종료

In [20]:
# 데이터프레임 생성 및 
df_mycafe = pd.DataFrame(cafe_kakaoMap_mylist, columns=['name', 'addr_load', 'rating_score', 'rating_num'])

# csv파일 저장
df_mycafe.to_csv("cafe_kakaoMap_mylist.csv", index=False, encoding='utf-8-sig')

In [ ]:
df

In [42]:
df_mycafe = pd.read_csv('cafe_kakaoMap_mylist.csv')
df_cafe = pd.read_csv('cafe_kakaoMap.csv')

In [43]:
df_mycafe["note"] = "editorPick"

In [ ]:
df_mycafe

In [ ]:
# 병합 데이터프레임 생성
df = df_cafe.merge(df_mycafe[['name', 'addr_load', 'note']], on='name', how='outer', suffixes=('_cafe', '_myCafe'))

# 병합 컬럼 생성
df['addr_load'] = df['addr_load_cafe'].fillna(df['addr_load_myCafe'])
df.drop(columns=['addr_load_cafe', 'addr_load_myCafe'], inplace=True)

In [49]:
# csv파일 저장
df.to_csv("cafe_kakaoMap_addMyList.csv", index=False, encoding='utf-8-sig')